In [1]:
import neuroglancer
import cloudvolume
import pandas as pd

## README
This example assumes you have precomputed datasets already made and hosted with cloudvolume.
To host the raw data in this example that I already converted to precomputed format, open an ipython terminal and do:
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/20161205_tp_bl6_lob45_1000r_01/647')
vol.viewer(port=1337) # You can choose the port here, but remember it for later
```
The last command will cause the window to hang -- that is the expected behavior. Do not try to do this in this jupyter notebook as it will cause your session to hang and you won't be able to run any of the following steps.

## Set the neuroglancer client you want to use. 
Default is Seung lab's which should work fine for this example

In [2]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

# Load in the raw data into neuroglancer and generate the link

In [ ]:
# This volume handle can be used to notify the viewer that the data has changed.
viewer = neuroglancer.Viewer()

# Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume
with viewer.txn() as s:
    s.layers['20161205_tp_bl6_lob45_1000r_01'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 20.0;
  emitRGBA(vec4(v, 0.0, 0.0, v));
}
'''                                                 
    )
print(viewer) # Click the link that is generated to see your volume displayed in Neuroglancer

## Host the raw-space allen atlas cloudvolume

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/20161205_tp_bl6_lob45_1000r_01/atlas')
vol.viewer(port=1338) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

## Load in the atlas
Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume using the same viewer object that you already made. If you go back to the neuroglancer window after running this it should be there in a new layer

In [4]:
with viewer.txn() as s:
    s.layers['rawatlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )

Use the look-up table to make a key binding to the Neuroglancer session.

In [6]:
# First you need to create a dictionary mapping the atlas id to the region name. 
# Here's my example

csv_file = '/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts_16bit.xlsx'

df = pd.read_excel(csv_file)

atlas_df = df.copy()

atlas_dict = {}
for ind, row in atlas_df.iterrows():
    atlas_dict[int(row['id'])] = row['name']
    
# Here is the actual code for making the key binding. Copy and paste this
# into your jupyter notebook where you have already made the viewer() object

num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['rawatlas']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                    (region_id, region_name))

    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'

Take a screenshot:

In [ ]:
ss = neuroglancer.ScreenshotSaver(viewer, '/home/wanglab/Documents/neuroglancer/screenshots')
ss.capture()   